In [64]:
import importlib
import codecs
import os
import pandas
import numpy 
import datetime as dt
import scipy.stats
import pandasql.sqldf
from IPython.core.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pandas.options.display.max_rows = 200
pandas.options.display.max_columns = 50
pandas.options.display.width = 300
pandas.set_option('display.float_format', lambda x: '%.8f' % x)

import trm3.impute
import trm3.utility

input_file = r"D:\projects\Python36-Package-Development\trm3pylib\data-csv\zillow_train.csv"
with codecs.open(input_file, 'r', encoding='utf-8') as f:
   df= pandas.read_csv(f, sep=",", header=0, low_memory=False, parse_dates=True)

df["datecol"] = pandas.to_datetime(df.transactiondate)
df["interval"] = df["datecol"] - dt.datetime.now().date()
df["fips"].fillna(value=df["fips"].mode().tolist()[0], inplace=True)


In [65]:
importlib.reload(trm3.impute)
importlib.reload(trm3.utility)

<module 'trm3.impute' from 'D:\\projects\\Python36-Package-Development\\trm3pylib\\trm3\\impute.py'>

<module 'trm3.utility' from 'D:\\projects\\Python36-Package-Development\\trm3pylib\\trm3\\utility.py'>

In [66]:
sdf = trm3.utility.df_contents(df, display_html_output=True)
#                   excel_filename=r"D:\projects\Python36-Package-Development\temp\test01.xlsx",
#                   excel_sheet_name="DF Summary")

,column_dtype,n_missing,n_non_missing,n_boolean,n_float,n_integer,n_string,n_datetime,n_timedelta,n_unique_values,mode,mean,stdev,min,p1,p5,p10,p25,median,p75,p90,p95,p99,max
Column Name,,,,,,,,,,,,,,,,,,,,,,,,
airconditioningtypeid,float64,61934,28748,0,28748,0,0,0,0,7,[1.0],1.81522193,2.97210778,1.00000000,1.00000000,1.00000000,1.00000000,1.00000000,1.00000000,1.00000000,1.00000000,13.00000000,13.00000000,13.00000000
architecturalstyletypeid,float64,90422,260,0,260,0,0,0,0,7,[7.0],7.23076923,2.72139659,2.00000000,2.00000000,3.00000000,7.00000000,7.00000000,7.00000000,7.00000000,7.00000000,8.00000000,21.00000000,21.00000000
assessmentyear,float64,532,90150,0,90150,0,0,0,0,2,[2015.0],2015.00000000,0.00000000,2015.00000000,2015.00000000,2015.00000000,2015.00000000,2015.00000000,2015.00000000,2015.00000000,2015.00000000,2015.00000000,2015.00000000,2015.00000000
basementsqft,float64,90639,43,0,43,0,0,0,0,40,[1528.0],713.58139535,437.43419818,100.00000000,126.04000000,169.60000000,196.40000000,407.50000000,616.00000000,872.00000000,1492.40000000,1528.00000000,1553.32000000,1555.00000000
bathroomcnt,float64,532,90150,0,90150,0,0,0,0,24,[2.0],2.27954520,1.00413283,0.00000000,0.00000000,1.00000000,1.00000000,2.00000000,2.00000000,3.00000000,3.00000000,4.00000000,5.50000000,20.00000000
bedroomcnt,float64,532,90150,0,90150,0,0,0,0,18,[3.0],3.03193566,1.15611372,0.00000000,0.00000000,1.00000000,2.00000000,2.00000000,3.00000000,4.00000000,4.00000000,5.00000000,6.00000000,16.00000000
buildingclasstypeid,float64,90666,16,0,16,0,0,0,0,2,[4.0],4.00000000,0.00000000,4.00000000,4.00000000,4.00000000,4.00000000,4.00000000,4.00000000,4.00000000,4.00000000,4.00000000,4.00000000,4.00000000
buildingqualitytypeid,float64,33398,57284,0,57284,0,0,0,0,9,[7.0],5.56539348,1.90044284,1.00000000,1.00000000,4.00000000,4.00000000,4.00000000,7.00000000,7.00000000,7.00000000,7.00000000,10.00000000,12.00000000
calculatedbathnbr,float64,1708,88974,0,88974,0,0,0,0,23,[2.0],2.30917459,0.97613689,1.00000000,1.00000000,1.00000000,1.00000000,2.00000000,2.00000000,3.00000000,3.00000000,4.00000000,5.50000000,20.00000000


In [5]:
df.columns.tolist()

['parcelid',
 'airconditioningtypeid',
 'architecturalstyletypeid',
 'basementsqft',
 'bathroomcnt',
 'bedroomcnt',
 'buildingclasstypeid',
 'buildingqualitytypeid',
 'calculatedbathnbr',
 'decktypeid',
 'finishedfloor1squarefeet',
 'calculatedfinishedsquarefeet',
 'finishedsquarefeet12',
 'finishedsquarefeet13',
 'finishedsquarefeet15',
 'finishedsquarefeet50',
 'finishedsquarefeet6',
 'fips',
 'fireplacecnt',
 'fullbathcnt',
 'garagecarcnt',
 'garagetotalsqft',
 'hashottuborspa',
 'heatingorsystemtypeid',
 'latitude',
 'longitude',
 'lotsizesquarefeet',
 'poolcnt',
 'poolsizesum',
 'pooltypeid10',
 'pooltypeid2',
 'pooltypeid7',
 'propertycountylandusecode',
 'propertylandusetypeid',
 'propertyzoningdesc',
 'rawcensustractandblock',
 'regionidcity',
 'regionidcounty',
 'regionidneighborhood',
 'regionidzip',
 'roomcnt',
 'storytypeid',
 'threequarterbathnbr',
 'typeconstructiontypeid',
 'unitcnt',
 'yardbuildingsqft17',
 'yardbuildingsqft26',
 'yearbuilt',
 'numberofstories',
 'firep

In [99]:
importlib.reload(trm3.impute)    
my_imputer = trm3.impute.Imputer(train_df = df,
                                 test_df = None,
                                 columns = [x for x in df.columns.tolist() if x[0:3] != "dv_" and x !="fips"],
#                                  columns = ["datecol"],
                                 groupby = ["fips"],
                                 label = "dv_avg_error",
                                 method = 2,
                                 mode_fallback_distinct_values = 3,
                                 mean_label_min_obs = .005,
                                 mean_label_bins = 10,
                                 nominal_new_category_min_obs = .01,
                                 missing_indicators = False,
                                 missing_indicators_min_obs = .01,
                                 model_x_columns = None,
                                 verbose=False
                                )

my_imputer_rules = my_imputer.fit()
my_imputer_rules.impute(impute_df=df, inplace=False, verbose=True)

<module 'trm3.impute' from 'D:\\projects\\Python36-Package-Development\\trm3pylib\\trm3\\impute.py'>

D:\applications\Anaconda3\lib\site-packages\pandas\core\groupby.py:3154: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  inc = np.r_[1, val[1:] != val[:-1]]


[001] parcelid
[002] airconditioningtypeid
[003] architecturalstyletypeid
[004] basementsqft
[005] bathroomcnt
[006] bedroomcnt
[007] buildingclasstypeid
[008] buildingqualitytypeid
[009] calculatedbathnbr
[010] decktypeid
[011] finishedfloor1squarefeet
[012] calculatedfinishedsquarefeet
[013] finishedsquarefeet12
[014] finishedsquarefeet13
[015] finishedsquarefeet15
[016] finishedsquarefeet50
[017] finishedsquarefeet6
[018] fireplacecnt
[019] fullbathcnt
[020] garagecarcnt
[021] garagetotalsqft
[022] hashottuborspa
[023] heatingorsystemtypeid
[024] latitude
[025] longitude
[026] lotsizesquarefeet
[027] poolcnt
[028] poolsizesum
[029] pooltypeid10
[030] pooltypeid2
[031] pooltypeid7
[032] propertycountylandusecode
[033] propertylandusetypeid
[034] propertyzoningdesc
[035] rawcensustractandblock
[036] regionidcity
[037] regionidcounty
[038] regionidneighborhood
[039] regionidzip
[040] roomcnt
[041] storytypeid
[042] threequarterbathnbr
[043] typeconstructiontypeid
[044] unitcnt
[045] ya

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,...,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,logerror,transactiondate,dv_min_error,dv_max_error,dv_avg_error,dv_first_error,dv_last_error,dv_transactions,last_error,datecol,interval
0,10711738,1.00000000,0.00000000,616.00000000,3.00000000,4.00000000,0.00000000,4.00000000,3.00000000,66.00000000,1244.00000000,2538.00000000,2538.00000000,1440.00000000,2104.00000000,1248.00000000,1986.00000000,6037.00000000,1.00000000,3.00000000,0.00000000,1.00000000,True,2.00000000,342.00000000,...,1.00000000,260.00000000,159.00000000,1978.00000000,1.00000000,[nan],245180.00000000,567112.00000000,2015.00000000,321932.00000000,7219.18000000,[nan],14.00000000,60371132000000.00000000,0.02760000,2016-08-02,0.02760000,0.02760000,0.02760000,0.02760000,0.02760000,1,0.00000000,2016-08-02,-429 days
1,10711755,1.00000000,0.00000000,616.00000000,3.00000000,3.00000000,0.00000000,4.00000000,3.00000000,66.00000000,1244.00000000,1589.00000000,1589.00000000,1440.00000000,2104.00000000,1248.00000000,1986.00000000,6037.00000000,1.00000000,3.00000000,0.00000000,1.00000000,True,2.00000000,342.00000000,...,1.00000000,260.00000000,159.00000000,1959.00000000,1.00000000,[nan],254691.00000000,459844.00000000,2015.00000000,205153.00000000,6901.09000000,[nan],14.00000000,60371132000000.00000000,-0.01820000,2016-08-02,-0.01820000,-0.01820000,-0.01820000,-0.01820000,-0.01820000,1,0.00000000,2016-08-02,-429 days
2,10711805,1.00000000,0.00000000,616.00000000,2.00000000,3.00000000,0.00000000,4.00000000,2.00000000,66.00000000,1244.00000000,2411.00000000,2411.00000000,1440.00000000,2104.00000000,1248.00000000,1986.00000000,6037.00000000,1.00000000,2.00000000,0.00000000,1.00000000,True,2.00000000,342.00000000,...,1.00000000,260.00000000,159.00000000,1973.00000000,1.00000000,[nan],235114.00000000,384787.00000000,2015.00000000,149673.00000000,4876.61000000,[nan],14.00000000,60371132000000.00000000,-0.10090000,2016-05-03,-0.10090000,-0.10090000,-0.10090000,-0.10090000,-0.10090000,1,0.00000000,2016-05-03,-520 days
3,10711816,1.00000000,0.00000000,616.00000000,2.00000000,4.00000000,0.00000000,4.00000000,2.00000000,66.00000000,1244.00000000,2232.00000000,2232.00000000,1440.00000000,2104.00000000,1248.00000000,1986.00000000,6037.00000000,1.00000000,2.00000000,0.00000000,1.00000000,True,2.00000000,342.00000000,...,1.00000000,260.00000000,159.00000000,1973.00000000,1.00000000,[nan],262309.00000000,437176.00000000,2015.00000000,174867.00000000,5560.07000000,[nan],14.00000000,60371132000000.00000000,-0.01210000,2016-04-05,-0.01210000,-0.01210000,-0.01210000,-0.01210000,-0.01210000,1,0.00000000,2016-04-05,-548 days
4,10711858,1.00000000,0.00000000,616.00000000,2.00000000,4.00000000,0.00000000,4.00000000,2.00000000,66.00000000,1244.00000000,1882.00000000,1882.00000000,1440.00000000,2104.00000000,1248.00000000,1986.00000000,6037.00000000,1.00000000,2.00000000,0.00000000,1.00000000,True,2.00000000,342.00000000,...,1.00000000,260.00000000,159.00000000,1973.00000000,1.00000000,[nan],232037.00000000,382055.00000000,2015.00000000,150018.00000000,4878.25000000,[nan],14.00000000,60371132000000.00000000,-0.04810000,2016-07-15,-0.04810000,-0.04810000,-0.04810000,-0.04810000,-0.04810000,1,0.00000000,2016-07-15,-447 days
5,10711910,1.00000000,0.00000000,616.00000000,2.00000000,3.00000000,0.00000000,4.00000000,2.00000000,66.00000000,1244.00000000,1477.00000000,1477.00000000,1440.00000000,2104.000

In [101]:
print(my_imputer_rules.python_code())

def impute_missing_values(df):

    #-----------------------------------------------------------------------
    # Function to fill missing for a specific column and combination of
    # groupby values.
    #-----------------------------------------------------------------------
    def gb_fillna(gb_fill_df, impute_col, expr, impute_val):
        _map = gb_fill_df.eval(expr)
        gb_fill_df.loc[_map, impute_col] = gb_fill_df.loc[_map, impute_col].fillna(value=impute_val, inplace=False)

    #-----------------------------------------------------------------------
    # Replace any missing values in the groupby variables.
    #-----------------------------------------------------------------------
    df['fips'].fillna(6037.0, inplace=True))



    #-----------------------------------------------------------------------
    # parcelid Imputation
    #-----------------------------------------------------------------------
    gb_fillna(impute_df if inplace else new_df,
             'pa

In [61]:
len(df)

90682

In [80]:
print("[%03d]" % 5)

[005]


In [111]:
cols = df.columns.tolist()
cols.sort()